In [2]:
import numpy as np

path = '/home/yuxuan/project/NeuralSurfaceField/Data/BuFF/buff_release_rot_const/sequences/00096/shortlong_hips/shortlong_hips_000216_cano.npy'

cano = np.load(path, allow_pickle=True).item()
cano_verts = cano['cano_points']
cano_normals = cano['cano_normals']

In [3]:
import open3d as o3d

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(cano_verts)
pcd.normals = o3d.utility.Vector3dVector(cano_normals)

# write point cloud
o3d.io.write_point_cloud("visualization/00096_shortlong_hips_000216_cano.ply", pcd)

True

In [1]:
from dataloaders.dataloader_buff import DataLoader_Buff_depth

splt_file = '/home/yuxuan/project/NeuralSurfaceField/assets/data_split/buff_male_train_val.pkl' 
subject_index_dict = {}
subject_index_dict.update({"00032_shortlong": 0,
                            "00096_shortlong": 1})
train_dataset = DataLoader_Buff_depth(mode='train', batch_size=2, num_workers=4, split_file=splt_file, subject_index_dict=subject_index_dict)
train_loader = train_dataset.get_loader()


Loading train data...


100%|██████████| 569/569 [00:04<00:00, 137.15it/s]

Data loaded, in total 569 train examples.



In [2]:
for i, batch in enumerate(train_loader):
    break

In [9]:
import open3d as o3d

# write function which uses open3d to write point cloud
def write_pcd(path, points, normals):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.normals = o3d.utility.Vector3dVector(normals)
    o3d.io.write_point_cloud(path, pcd)

path_cano = '/home/yuxuan/project/NeuralSurfaceField/visualization/cano.ply'
path_posed = '/home/yuxuan/project/NeuralSurfaceField/visualization/posed.ply'

# write cano point cloud
write_pcd(path_cano, batch['cano_points'][0].cpu().numpy(), batch['cano_normals'][0].cpu().numpy())
# write posed point cloud
write_pcd(path_posed, batch['scan_points'][0].cpu().numpy().transpose(), batch['scan_normals'][0].cpu().numpy().transpose())

In [5]:
import open3d as o3d
import numpy as np

dict_ = np.load('/home/yuxuan/project/NeuralSurfaceField/Data_scan/BuFF/buff_release/sequences/00159/shortshort_twist_tilt_left/shortshort_twist_tilt_left.000050.npz')

# o3d point cloud visualization
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(dict_['v_posed'])
o3d.visualization.draw_geometries([pcd])

In [1]:
import pickle as pkl

split_file = '/mnt/qb/work/ponsmoll/yxue80/project/NeuralSurfaceField/assets/data_split/buff_male_train_val.pkl'

with open(split_file, "rb") as f:
    split = pkl.load(f)

In [2]:
from libs.global_variable import ROOT_DIR
import os

split_org = {
    'train': [],
    'val': []
}

for subj_train in split['train']:
    if subj_train.split('/')[5].startswith('shortlong'):
        split_org['train'].append(subj_train)


for subj_val in split['val']:
    if subj_val.split('/')[5].startswith('shortlong'):
        split_org['val'].append(subj_val)


pkl.dump(split_org, open(split_file.replace('buff_male_train_val.pkl', 'buff_male_shapefusion.pkl'), 'wb'))

In [3]:
batch['cano_points'].shape

torch.Size([2, 30000, 3])

In [3]:
import os
import torch

# load old checkpoint
shapefusion_path = '/mnt/qb/work/ponsmoll/yxue80/project/shapefusion/experiments/PoseImplicit_exp_id_808'
shapefusion_checkpoint_path = os.path.join(shapefusion_path, 'checkpoints', 'checkpoint_epoch_2150.tar')

print('Loaded checkpoint from: {}'.format(shapefusion_checkpoint_path))
checkpoint = torch.load(shapefusion_checkpoint_path)

print(checkpoint.keys())

# save to new checkpoint
path = 'checkpoint_epoch_{}.tar'.format(checkpoint['epoch'])
if not os.path.exists(path):
    model_weights = {'epoch': checkpoint['epoch'],
                    'optimizer_state_dict': checkpoint['optimizer_state_dict'], 
                    'scheduler_state_dict': checkpoint['scheduler_state_dict'],
                    'feat_optimizer_state_dict': checkpoint['feat_optimizer_state_dict'],
                    'feat_scheduler_state_dict': checkpoint['feat_scheduler_state_dict'],
                    'pose_encoder_state_dict': checkpoint['pose_encoder_state_dict'],
                    'nsf_decoder_state_dict': checkpoint['shape_geometry_decoder_state_dict']}
    torch.save(model_weights, path)

Loaded checkpoint from: /mnt/qb/work/ponsmoll/yxue80/project/shapefusion/experiments/PoseImplicit_exp_id_808/checkpoints/checkpoint_epoch_2150.tar
dict_keys(['epoch', 'optimizer_state_dict', 'scheduler_state_dict', 'feat_optimizer_state_dict', 'feat_scheduler_state_dict', 'pose_encoder_state_dict', 'shape_geometry_decoder_state_dict', 'conditional_ndf_state_dict'])
